In [2]:
!date
!python --version

Thu May  8 03:48:57 AM UTC 2025
Python 3.11.12


# BoWとword2vec用の前処理
分かち書きに時間がかかる（當間環境で約20分）ため、実験コードを実行し直す都度分かち書きからやり直すのは時間がもったいない。そこでここでは分かち書きだけを行い、結果を（[pickle形式](https://docs.python.org/ja/3.13/library/pickle.html)）ファイルとして保存しておく。機械学習する側のコードでは、このファイルを読み込むだけの時間で実験を再開することができるようになる。

本コードを実行すると、以下のファイルに前処理済みデータが保存される。
- 学習データ: `prepcoecced_train.pkl`
- 検証データ: `prepcoecced_val.pkl`
- テストデータ: `prepcoecced_test.pkl`

pickleファイルの中身は「1件のデータが辞書型」であり、それをリストとして保存している。`sent1`は前提文、`sent2`は仮説文。`bow_input`がBoW用の分かち書き結果。`w2v_input`がword2vec用の分散ベクトル。`label`が教師ラベル（文字列）である。
```Python
[
  {
    "sent1": "男の子が水たまりで遊んでいる。",
    "sent2": "子供が遊んでいる。",
    "bow_input": "男の子 が 水たまり で 遊ん で いる 。 [SEP] 子供 が 遊ん で いる 。",
    "w2v_input": np.array([...]),  # shape: (600,)
    "label": "entailment"
  },
  ...
]
```

実行後、作成されるファイルをダウンロードするのを忘れないようにしよう。忘れてしまったらまたこれを実行し直すことになります。

## 前提
- [Github: JGLUE/datasets/jnil-v1.3/](https://github.com/yahoojapan/JGLUE/tree/main/datasets/jnli-v1.3) にある以下のファイルをダウンロードしておき、本ノートブックを実行する前にアップロードしておいてください。
    - train-v1.3.json（学習用データセット）
    - val-v1.3.json（検証用データセット）
    - test-v1.3.json（テスト用データセット）


In [3]:
# spacyインストール
!pip install ginza ja_ginza

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.1/72.1 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 63.3 MB/s eta 0:00:00


In [4]:
# ====================================
# ✅ 必要ライブラリ
# ====================================
import spacy
import pickle
import os
from tqdm import tqdm
import pandas as pd
import numpy as np

# ====================================
# ✅ GiNZAのロード（時間がかかります）
# ====================================
nlp = spacy.load("ja_ginza")

# ====================================
# ✅ 入力ファイルの読み込み
# ====================================
def load_jsonl(filepath):
    with open(filepath, encoding="utf-8") as f:
        return [eval(line.strip()) for line in f]

def load_and_prepare(file_path):
    data = load_jsonl(file_path)
    return [(ex["sentence1"], ex["sentence2"], ex["label"]) for ex in data]

train_data = load_and_prepare("train-v1.3.json")
val_data = load_and_prepare("valid-v1.3.json")
test_data = load_and_prepare("test-v1.3.json")

# ====================================
# ✅ 文をspaCyで処理して保存
# ====================================
def preprocess_and_save(data, output_file):
    results = []
    for sent1, sent2, label in tqdm(data):
        # GiNZAで処理
        doc1 = nlp(sent1)
        doc2 = nlp(sent2)

        # BoW用分かち書き（空白区切り文字列）
        tokens1 = [t.text for t in doc1 if not t.is_space]
        tokens2 = [t.text for t in doc2 if not t.is_space]
        bow_joined = " ".join(tokens1 + ["[SEP]"] + tokens2)

        # word2vec用ベクトル平均
        vecs1 = [t.vector for t in doc1 if t.has_vector and not t.is_space]
        vecs2 = [t.vector for t in doc2 if t.has_vector and not t.is_space]
        if vecs1:
            vec1_avg = np.mean(vecs1, axis=0)
        else:
            vec1_avg = np.zeros(nlp.vocab.vectors_length)
        if vecs2:
            vec2_avg = np.mean(vecs2, axis=0)
        else:
            vec2_avg = np.zeros(nlp.vocab.vectors_length)
        vec_concat = np.concatenate([vec1_avg, vec2_avg])

        results.append({
            "sent1": sent1, # 前提文
            "sent2": sent2, # 仮説文
            "bow_input": bow_joined, # BoW用の分かち書きした文（[SEP]付きで結合）
            "w2v_input": vec_concat, # word2vec用の平均ベクトトル（2文の平均ベクトルを結合）
            "label": label
        })

    # pickle保存
    with open(output_file, "wb") as f:
        pickle.dump(results, f)

# ====================================
# ✅ 実行：train / val / test それぞれ保存
# ====================================
preprocess_and_save(train_data, "preprocessed_train.pkl")
preprocess_and_save(val_data, "preprocessed_val.pkl")
preprocess_and_save(test_data, "preprocessed_test.pkl")

100%|██████████| 2508/2508 [01:53<00:00, 22.12it/s]


## 実行終了後、3つのpklファイルをダウンロードすること